In [ ]:
from transformers import AutoFeatureExtractor,AutoModelForImageClassification,BeitFeatureExtractor, BeitModel
from transformers import AutoFeatureExtractor, AutoModelForPreTraining
import torch
from datasets import load_dataset
import numpy as np
from annoy import AnnoyIndex
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# 加载专利数据集
import os
def require_filename(path):
    root = path
    file_list = []
    for dirpath, dirnames, filenames in os.walk(root):
        for filepath in filenames:
            file_list.append(os.path.join(dirpath, filepath))
    return file_list
file_list1 = require_filename('/home/jovyan/mnt/juicefs-share-bigdata-dip-ai/NLP/patent_pic/wgpic1')
file_list = require_filename('/home/jovyan/mnt/juicefs-share-bigdata-dip-ai/NLP/patent_pic/wgpic')

In [ ]:
# 加载数据集
dataset = load_dataset("cifar10")
# image = dataset["test"]["image"][0]

In [ ]:
# image = dataset['test']['img'][0]
test_label = dataset['test']['label']
dataset['test']['image'][1]

In [ ]:
#加载模型测试代码
# feature_extractor = BeitFeatureExtractor.from_pretrained("microsoft/beit-base-patch16-224-pt22k")
# model = BeitModel.from_pretrained("microsoft/beit-base-patch16-224-pt22k")
# model.cuda()

In [ ]:
def image_to_vec(image):
    ls = {}
    inputs = feature_extractor(image, return_tensors="pt")
    inputs = inputs.to('cuda')
    with torch.no_grad():
        hidden_states = model(**inputs, return_dict=True, output_hidden_states=True).hidden_states
        
        output_hidden_state = (hidden_states[-1] + hidden_states[1]).mean(dim=1)
        vec = output_hidden_state.cpu().numpy()[0]
        ls['first_last_avg'] = vec
        
        output_hidden_state = (hidden_states[-1]).mean(dim=1)
        vec = output_hidden_state.cpu().numpy()[0]
        ls['last_avg'] = vec
        
        output_hidden_state = (hidden_states[-1] + hidden_states[-2]).mean(dim=1)
        vec = output_hidden_state.cpu().numpy()[0]
        ls['last2avg'] = vec
        
        output_hidden_state = (hidden_states[-1])[:, 0, :]
        vec = output_hidden_state.cpu().numpy()[0]
        ls['cls'] = vec
        
    return ls
# 降维函数
def compute_kernel_bias(vecs):
    """计算kernel和bias
    最后的变换：y = (x + bias).dot(kernel)
    """
#     vecs_ = np.concatenate(vecs, axis=0)
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(1 / np.sqrt(s)))
    return W, -mu

def transform_and_normalize(vecs, kernel=None, bias=None, dim=[]):
    """应用变换，然后标准化
    """
    def std1(vecs_):
        norms = (vecs_**2).sum(axis=1, keepdims=True)**0.5
        return vecs_ / np.clip(norms, 1e-8, np.inf)
    
    res = []
    if not (kernel is None or bias is None):
        vecs = (vecs + bias).dot(kernel)
#         res.append(std1(vecs)[0].round(6))
        return vecs.round(6)
# 

In [ ]:
a = image_to_vec(image)

In [ ]:
model_name_ls

cifar10计算向量，保存检索模型代码

In [ ]:
f = 768  # 句向量维度
# 
# 模型保存地址
model_name_ls = []
for model_name in ["microsoft/beit-base-patch16-224-pt22k"]:
    # 加载模型
    vec_ls = []
    feature_extractor = BeitFeatureExtractor.from_pretrained(model_name)
    model = BeitModel.from_pretrained(model_name)
    model.cuda()
    # 计算向量
    for image_ in tqdm(dataset['test']['img']):
        vec_ = image_to_vec(image_)
        vec_ls.append(vec_)
    for i in tqdm(['first_last_avg', 'last_avg', 'last2avg','cls']):
        ls_vecs = [_[i] for _ in vec_ls]
        #计算降维矩阵
        ls_vecs_ = np.array(ls_vecs)
        kernel, bias = compute_kernel_bias(ls_vecs_)
        #分别构建检索模型
        for j in [768, 128]:
            m = 0
            name_ = '%s_cifar10_%s_%s.ann' %(model_name.split('/')[1].split('-')[0], i, j)     
            t_768 = AnnoyIndex(j)
            t_768.on_disk_build(name_)
            for vec_ in transform_and_normalize(ls_vecs_, kernel[:, :j], bias, []):
                t_768.add_item(m, vec_)
                m+=1
            t_768.build(10)         
#             model_name_ls.append(name_)

   

专利图片计算向量，保存相似模型代码

In [ ]:
f = 768  # 句向量维度
file_ls = []
# 模型保存地址
model_name_ls = []
# 测试代码，支持同时测试多个模型，多种向量降维长度，多种取向量方式
# for model_name in ["microsoft/beit-base-patch16-224-pt22k"]:
for model_name in ["facebook/vit-mae-base"]:    
    # 加载模型
    vec_ls = []
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
    model = AutoModelForPreTraining.from_pretrained(model_name)
    model.cuda()
    # 计算向量
    for file in tqdm(file_list):
        try:
            image = Image.open(file)
            image = image.resize((224, 224))  
            vec_ = image_to_vec(image)
        except:
            continue
#             image = Image.open(file).convert('RGB')
#             image = image.resize((224, 224))
        file_ls.append(file)
        
        vec_ls.append(vec_)
    for i in tqdm(['first_last_avg', 'last_avg', 'last2avg','cls']):
        ls_vecs = [_[i] for _ in vec_ls]
        #计算降维矩阵
        ls_vecs_ = np.array(ls_vecs)
        kernel, bias = compute_kernel_bias(ls_vecs_)
        #分别构建检索模型
        for j in [768, 128]:
            m = 0
            name_ = '%s_patent_%s_%s.ann' %(model_name.split('/')[1].split('-')[1], i, j)     
            t_768 = AnnoyIndex(j)
            t_768.on_disk_build(name_)
            for vec_ in transform_and_normalize(ls_vecs_, kernel[:, :j], bias, []):
                t_768.add_item(m, vec_)
                m+=1
            t_768.build(10)         
#             model_name_ls.append(name_)

map_dict = {}
m = 0
for i in file_ls:
    map_dict[m] = i
    m+=1

In [ ]:
sim = t_768.get_nns_by_item(0, 2000,  include_distances=True)    
score = [round(1-i**2/2, 4) for i in sim[1]]
sim_label = [test_label[i] for i in sim[0]]
sim_label.count(3)

In [ ]:
# 取所有保存的检索模型
model = require_filename('./')
# model = [i for i in model if 'patent' in i]
# 只取mae模型的结果
model = [i for i in model if 'mae' in i]

In [ ]:
#加载需要测试的模型，保存在列表
model_ls = []
for model_name in model:
    f = int(model_name[-7:-4])
    t_384 = AnnoyIndex(f)
    t_384.load(model_name)
    model_ls.append(t_384)
#     break


In [ ]:
# ./beit_patent_first_last_avg_128.ann
# ./beit_patent_last_avg_128.ann

In [ ]:
#3456，6549， 4354， 7346
id_ = 4354
for t_768, model_name in zip(model_ls, model):
    print(model_name)
    sim = t_768.get_nns_by_item(id_, 9,  include_distances=True)    
        # 相似模型返回的distances是欧式距离，不方便观察相似性，将欧式距离转化为余弦相似度
    sim = [[map_dict[i], str(round(1-j**2/2, 4))] for i, j in zip(sim[0], sim[1])]
    m = 1
    plt.figure()
    for i in sim:
        image = Image.open(i[0])
        plt.subplot(3, 3, m)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        plt.title(str(i[1]))
        m+=1